In [1]:
# %% [markdown]
# # Exercise 3: Improve the Risk Model
# 
# Goal: Improve upon the baseline GLM from the glum tutorial using:
# 1. Better train/test splitting
# 2. Feature engineering (splines)
# 3. Gradient boosting (LightGBM)

# %%
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dask_ml.preprocessing import Categorizer
from glum import GeneralizedLinearRegressor, TweedieDistribution
from lightgbm import LGBMRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, SplineTransformer, StandardScaler

from ps3.data import create_sample_column, load_transform

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# %% [markdown]
# ## Task 1 & 2: Load Data and Create Train/Test Split

# %%
# Load the French motor insurance data
df = load_transform()

print(f"Data shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Data shape: (678013, 14)

First few rows:


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,ClaimAmountCut
0,1,0,0.10,D,5,0,5,50,B12,Regular,1217,R82,0.0,0.0
1,3,0,0.77,D,5,0,5,50,B12,Regular,1217,R82,0.0,0.0
2,5,0,0.75,B,6,1,5,50,B12,Diesel,54,R22,0.0,0.0
3,10,0,0.09,B,7,0,4,50,B12,Diesel,76,R72,0.0,0.0
4,11,0,0.84,B,7,0,4,50,B12,Diesel,76,R72,0.0,0.0


In [3]:
# %%
# Use our deterministic sample split function from Exercise 1
df = create_sample_column(df, columns='IDpol', training_frac=0.8)

# Check the split
print(f"Sample distribution:")
print(df['sample'].value_counts())
print(f"\nTrain proportion: {(df['sample'] == 'train').mean():.2%}")

Sample distribution:
sample
train    541985
test     136028
Name: count, dtype: int64

Train proportion: 79.94%


In [4]:
# %% [markdown]
# ## Task 3: Train Benchmark Tweedie Model
# 
# **Question: Why do we divide ClaimAmountCut by Exposure?**
# 
# **Answer:** We divide by Exposure to get the **Pure Premium** (claim amount per unit of exposure time).
# 
# - `ClaimAmountCut`: Total claim amount for a policy (in currency)
# - `Exposure`: The fraction of year the policy was active (0 to 1)
# - `PurePremium = ClaimAmountCut / Exposure`: Annualized claim cost
# 
# This normalization allows us to compare policies with different exposure periods fairly.
# A policy active for 0.5 years with €500 in claims has the same risk (€1000/year) 
# as a policy active for 1 year with €1000 in claims.

# %%
# Prepare target variable and weights
weight = df["Exposure"].values
df["PurePremium"] = df["ClaimAmountCut"] / df["Exposure"]
y = df["PurePremium"]

# Create train/test indices
train = np.where(df["sample"] == "train")
test = np.where(df["sample"] == "test")
df_train = df.iloc[train].copy()
df_test = df.iloc[test].copy()

print(f"Train set size: {len(df_train)}")
print(f"Test set size: {len(df_test)}")

Train set size: 541985
Test set size: 136028


In [ ]:
# Test if glum works
try:
    from glum import GeneralizedLinearRegressor, TweedieDistribution
    print("✓ glum imported")
    
    # Try creating a simple model
    TweedieDist = TweedieDistribution(1.5)
    test_model = GeneralizedLinearRegressor(family=TweedieDist)
    print("✓ glum model created successfully")
except Exception as e:
    print(f"✗ glum error: {e}")

    # had to do the following in terminal to fix glum installation and prevent kernel crash:
    # pip uninstall glum
    # conda install -c conda-forge glum

✓ glum imported
✓ glum model created successfully


In [6]:
# %%
# Define categorical and continuous features
categoricals = ["VehBrand", "VehGas", "Region", "Area", "DrivAge", "VehAge", "VehPower"]
predictors = categoricals + ["BonusMalus", "Density"]

# Prepare data for GLM
glm_categorizer = Categorizer(columns=categoricals)
X_train_t = glm_categorizer.fit_transform(df[predictors].iloc[train])
X_test_t = glm_categorizer.transform(df[predictors].iloc[test])
y_train_t, y_test_t = y.iloc[train], y.iloc[test]
w_train_t, w_test_t = weight[train], weight[test]

# Train Tweedie GLM (baseline)
TweedieDist = TweedieDistribution(1.5)
t_glm1 = GeneralizedLinearRegressor(family=TweedieDist, l1_ratio=1, fit_intercept=True)
t_glm1.fit(X_train_t, y_train_t, sample_weight=w_train_t)

print("Baseline GLM trained!")
print(f"Number of features: {len(t_glm1.feature_names_)}")

Baseline GLM trained!
Number of features: 59


In [7]:
# %%
# Make predictions
df_test["pp_t_glm1"] = t_glm1.predict(X_test_t)
df_train["pp_t_glm1"] = t_glm1.predict(X_train_t)

# Calculate deviance (lower is better)
train_deviance = TweedieDist.deviance(
    y_train_t, df_train["pp_t_glm1"], sample_weight=w_train_t
) / np.sum(w_train_t)

test_deviance = TweedieDist.deviance(
    y_test_t, df_test["pp_t_glm1"], sample_weight=w_test_t
) / np.sum(w_test_t)

print(f"Baseline GLM Results:")
print(f"  Training deviance: {train_deviance:.6f}")
print(f"  Testing deviance:  {test_deviance:.6f}")
print(f"  Overfit ratio (test/train): {test_deviance/train_deviance:.3f}")

print(f"\nTotal claim amount on test set:")
print(f"  Observed:  €{df['ClaimAmountCut'].values[test].sum():,.0f}")
print(f"  Predicted: €{np.sum(df['Exposure'].values[test] * t_glm1.predict(X_test_t)):,.0f}")

Baseline GLM Results:
  Training deviance: 74.096244
  Testing deviance:  72.419647
  Overfit ratio (test/train): 0.977

Total claim amount on test set:
  Observed:  €9,640,879
  Predicted: €10,021,806


In [8]:
# %% [markdown]
# ## Task 4: Improve Model with Splines
# 
# We'll add splines for BonusMalus and Density to capture non-linear relationships.

# %%
# Define numeric columns that will use splines
numeric_cols = ["BonusMalus", "Density"]

# Create a pipeline for numeric features:
# 1. StandardScaler: normalize the features
# 2. SplineTransformer: create flexible basis functions
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('spline', SplineTransformer(n_knots=5, degree=3, knots='quantile', include_bias=False))
])

# Combine numeric and categorical transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', OneHotEncoder(sparse_output=False, drop='first'), categoricals),
    ]
)

# Set output format to pandas for easier inspection
preprocessor.set_output(transform="pandas")

# Create full pipeline: preprocessing + GLM
model_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('estimate', GeneralizedLinearRegressor(family=TweedieDist, l1_ratio=1, fit_intercept=True))
])

print("Pipeline created!")
model_pipeline

Pipeline created!


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('spline',
                                                                   SplineTransformer(include_bias=False,
                                                                                     knots='quantile'))]),
                                                  ['BonusMalus', 'Density']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['VehBrand', 'VehGas',
                                                   'Region', 'Area', 'DrivAge',
                                                   'VehAge', 'VehPower'])])),
                ('estimate',
                 GeneralizedLinearRegressor(family=<glum._distribution.TweedieDistribution object at 0x10c27f610>,
                                            l1_ratio=1))])

In [9]:
# %%
# Let's check that the transforms work correctly
X_transformed = model_pipeline[:-1].fit_transform(df_train[predictors])
print(f"Transformed shape: {X_transformed.shape}")
print(f"Original features: {len(predictors)}")
print(f"Transformed features: {X_transformed.shape[1]}")
print(f"\nFirst few columns:")
print(X_transformed.columns[:10].tolist())

Transformed shape: (541985, 62)
Original features: 9
Transformed features: 62

First few columns:
['num__BonusMalus_sp_0', 'num__BonusMalus_sp_1', 'num__BonusMalus_sp_2', 'num__BonusMalus_sp_3', 'num__BonusMalus_sp_4', 'num__BonusMalus_sp_5', 'num__Density_sp_0', 'num__Density_sp_1', 'num__Density_sp_2', 'num__Density_sp_3']


In [10]:
# %%
# Train the improved GLM with splines
model_pipeline.fit(
    df_train[predictors], 
    y_train_t, 
    estimate__sample_weight=w_train_t
)

print("✓ Improved GLM with splines trained!")
print(f"Number of features: {len(model_pipeline[-1].feature_names_)}")

/opt/anaconda3/lib/python3.13/site-packages/glum/_solvers.py:59: ConvergenceWarning: Coordinate descent did not converge. You might want to increase the number of iterations. Minimum norm subgradient: 0.0009581057541339925, tolerance: 0.0001
  new_coef, gap, _, _, n_cycles = enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.13/site-packages/glum/_solvers.py:59: ConvergenceWarning: Coordinate descent did not converge. You might want to increase the number of iterations. Minimum norm subgradient: 0.0007394173511698353, tolerance: 0.0001
  new_coef, gap, _, _, n_cycles = enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.13/site-packages/glum/_solvers.py:59: ConvergenceWarning: Coordinate descent did not converge. You might want to increase the number of iterations. Minimum norm subgradient: 0.000569771231451657, tolerance: 0.0001
  new_coef, gap, _, _, n_cycles = enet_coordinate_descent_gram(
/opt/anaconda3/lib/python3.13/site-packages/glum/_solvers.py:59: ConvergenceWarning

✓ Improved GLM with splines trained!
Number of features: 62


In [11]:
# %%
# Make predictions with improved model
df_test["pp_t_glm2"] = model_pipeline.predict(df_test[predictors])
df_train["pp_t_glm2"] = model_pipeline.predict(df_train[predictors])

# Calculate deviance for improved model
train_deviance_glm2 = TweedieDist.deviance(
    y_train_t, df_train["pp_t_glm2"], sample_weight=w_train_t
) / np.sum(w_train_t)

test_deviance_glm2 = TweedieDist.deviance(
    y_test_t, df_test["pp_t_glm2"], sample_weight=w_test_t
) / np.sum(w_test_t)

print("=" * 60)
print("MODEL COMPARISON")
print("=" * 60)
print(f"\nBaseline GLM (no splines):")
print(f"  Training deviance: {train_deviance:.6f}")
print(f"  Testing deviance:  {test_deviance:.6f}")
print(f"  Overfit ratio:     {test_deviance/train_deviance:.3f}")

print(f"\nImproved GLM (with splines):")
print(f"  Training deviance: {train_deviance_glm2:.6f}")
print(f"  Testing deviance:  {test_deviance_glm2:.6f}")
print(f"  Overfit ratio:     {test_deviance_glm2/train_deviance_glm2:.3f}")

print(f"\nImprovement:")
print(f"  Train deviance reduction: {train_deviance - train_deviance_glm2:.6f} ({(train_deviance - train_deviance_glm2)/train_deviance*100:.2f}%)")
print(f"  Test deviance reduction:  {test_deviance - test_deviance_glm2:.6f} ({(test_deviance - test_deviance_glm2)/test_deviance*100:.2f}%)")

print(f"\nTotal claim amount on test set:")
print(f"  Observed:  €{df['ClaimAmountCut'].values[test].sum():,.0f}")
print(f"  GLM1 pred: €{np.sum(df['Exposure'].values[test] * df_test['pp_t_glm1']):,.0f}")
print(f"  GLM2 pred: €{np.sum(df['Exposure'].values[test] * df_test['pp_t_glm2']):,.0f}")

MODEL COMPARISON

Baseline GLM (no splines):
  Training deviance: 74.096244
  Testing deviance:  72.419647
  Overfit ratio:     0.977

Improved GLM (with splines):
  Training deviance: 73.705976
  Testing deviance:  72.252291
  Overfit ratio:     0.980

Improvement:
  Train deviance reduction: 0.390268 (0.53%)
  Test deviance reduction:  0.167356 (0.23%)

Total claim amount on test set:
  Observed:  €9,640,879
  GLM1 pred: €10,021,806
  GLM2 pred: €9,999,830


## Task 4: Answers to Questions

### Question 1: How does the deviance on the train and test set change?

**Answer:**

Adding splines for `BonusMalus` and `Density` improved the model performance:

- **Training deviance**: Decreased from 74.10 → 73.71 (0.53% improvement)
- **Testing deviance**: Decreased from 72.42 → 72.25 (0.23% improvement)

The splines allow the model to capture non-linear relationships between these features and the target, leading to better fit. The improvement is modest but consistent on both train and test sets.

**Total claims prediction also improved:**
- Baseline GLM: €10,021,806 (overestimated by 3.9%)
- Spline GLM: €9,999,830 (overestimated by 3.7%)
- The improved model is more accurate!

---

### Question 2: How could we check whether we are overfitting?

**Answer:**

We check for overfitting by comparing training vs testing performance:

**Method 1: Overfit Ratio (test deviance / train deviance)**
- Baseline GLM: 0.977 (test < train → slight underfit)
- Spline GLM: 0.980 (test ≈ train → good fit)
- **Interpretation**: Ratio close to 1.0 indicates good generalization. If ratio >> 1, we're overfitting.

**Method 2: Compare deviance reduction on train vs test**
- Train improvement: 0.53%
- Test improvement: 0.23%
- **Interpretation**: Both improved, so we're not overfitting. If train improved a lot but test got worse, that would indicate overfitting.

**Conclusion**: The spline model shows **no signs of overfitting**. In fact, both models have test deviance slightly lower than train deviance, suggesting we could potentially add more model complexity.


In [12]:
# %% [markdown]
# ## Task 5: Train LightGBM Regressor
# 
# Now let's try gradient boosting to see if we can improve beyond the parametric GLM.

# %%
# Create LightGBM model
# For Tweedie regression, we use objective='tweedie' with tweedie_variance_power=1.5
lgbm_model = LGBMRegressor(
    objective='tweedie',
    tweedie_variance_power=1.5,  # Same as our GLM
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    verbose=-1  # Suppress warnings
)

# Train the model
print("Training LightGBM...")
lgbm_model.fit(
    X_train_t, 
    y_train_t, 
    sample_weight=w_train_t
)

print("✓ LightGBM trained!")
print(f"Number of trees: {lgbm_model.n_estimators}")

Training LightGBM...
✓ LightGBM trained!
Number of trees: 100


In [13]:
# %%
# Make predictions
df_test["pp_t_lgbm"] = lgbm_model.predict(X_test_t)
df_train["pp_t_lgbm"] = lgbm_model.predict(X_train_t)

# Calculate deviance
train_deviance_lgbm = TweedieDist.deviance(
    y_train_t, df_train["pp_t_lgbm"], sample_weight=w_train_t
) / np.sum(w_train_t)

test_deviance_lgbm = TweedieDist.deviance(
    y_test_t, df_test["pp_t_lgbm"], sample_weight=w_test_t
) / np.sum(w_test_t)

print("=" * 60)
print("MODEL COMPARISON - ALL MODELS")
print("=" * 60)

print(f"\n1. Baseline GLM (no splines):")
print(f"   Training: {train_deviance:.6f}, Testing: {test_deviance:.6f}, Ratio: {test_deviance/train_deviance:.3f}")

print(f"\n2. Improved GLM (with splines):")
print(f"   Training: {train_deviance_glm2:.6f}, Testing: {test_deviance_glm2:.6f}, Ratio: {test_deviance_glm2/train_deviance_glm2:.3f}")

print(f"\n3. LightGBM (untuned):")
print(f"   Training: {train_deviance_lgbm:.6f}, Testing: {test_deviance_lgbm:.6f}, Ratio: {test_deviance_lgbm/train_deviance_lgbm:.3f}")

print(f"\nTotal claim predictions on test set:")
print(f"   Observed:  €{df['ClaimAmountCut'].values[test].sum():,.0f}")
print(f"   GLM1:      €{np.sum(df['Exposure'].values[test] * df_test['pp_t_glm1']):,.0f}")
print(f"   GLM2:      €{np.sum(df['Exposure'].values[test] * df_test['pp_t_glm2']):,.0f}")
print(f"   LGBM:      €{np.sum(df['Exposure'].values[test] * df_test['pp_t_lgbm']):,.0f}")

MODEL COMPARISON - ALL MODELS

1. Baseline GLM (no splines):
   Training: 74.096244, Testing: 72.419647, Ratio: 0.977

2. Improved GLM (with splines):
   Training: 73.705976, Testing: 72.252291, Ratio: 0.980

3. LightGBM (untuned):
   Training: 66.913447, Testing: 72.188278, Ratio: 1.079

Total claim predictions on test set:
   Observed:  €9,640,879
   GLM1:      €10,021,806
   GLM2:      €9,999,830
   LGBM:      €8,780,746


In [15]:
# %% [markdown]
# ### Tune LightGBM with GridSearchCV
# 
# We'll tune learning_rate and n_estimators to reduce overfitting.
# Lower learning rate with more trees often generalizes better.

# %%
# Define parameter grid (smaller grid to save time)
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
}

# Create base model
lgbm_base = LGBMRegressor(
    objective='tweedie',
    tweedie_variance_power=1.5,
    max_depth=5,
    random_state=42,
    verbose=-1
)

# GridSearchCV with 3-fold CV (NO n_jobs to avoid the error)
print("Starting GridSearchCV (this may take a few minutes)...")
grid_search = GridSearchCV(
    lgbm_base,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1
)

# Fit with sample weights
print("Training... (9 combinations × 3 folds = 27 models)")
grid_search.fit(X_train_t, y_train_t, sample_weight=w_train_t)

print("\n✓ GridSearch complete!")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score (neg MSE): {grid_search.best_score_:.2f}")

Starting GridSearchCV (this may take a few minutes)...
Training... (9 combinations × 3 folds = 27 models)
Fitting 3 folds for each of 9 candidates, totalling 27 fits

✓ GridSearch complete!
Best parameters: {'learning_rate': 0.01, 'n_estimators': 500}
Best CV score (neg MSE): -206512910.65


In [16]:
# %%
# Get best model and make predictions
best_lgbm = grid_search.best_estimator_
df_test["pp_t_lgbm_tuned"] = best_lgbm.predict(X_test_t)
df_train["pp_t_lgbm_tuned"] = best_lgbm.predict(X_train_t)

# Calculate deviance for tuned model
train_deviance_lgbm_tuned = TweedieDist.deviance(
    y_train_t, df_train["pp_t_lgbm_tuned"], sample_weight=w_train_t
) / np.sum(w_train_t)

test_deviance_lgbm_tuned = TweedieDist.deviance(
    y_test_t, df_test["pp_t_lgbm_tuned"], sample_weight=w_test_t
) / np.sum(w_test_t)

print("=" * 60)
print("FINAL MODEL COMPARISON")
print("=" * 60)

models = [
    ("Baseline GLM", train_deviance, test_deviance),
    ("GLM + Splines", train_deviance_glm2, test_deviance_glm2),
    ("LGBM (untuned)", train_deviance_lgbm, test_deviance_lgbm),
    ("LGBM (tuned)", train_deviance_lgbm_tuned, test_deviance_lgbm_tuned),
]

for name, train_dev, test_dev in models:
    ratio = test_dev / train_dev
    print(f"\n{name}:")
    print(f"  Train: {train_dev:.6f}, Test: {test_dev:.6f}, Ratio: {ratio:.3f}")

print(f"\n{'Model':<20} {'Test Deviance':<15} {'Rank'}")
print("-" * 45)
sorted_models = sorted(models, key=lambda x: x[2])  # Sort by test deviance
for rank, (name, _, test_dev) in enumerate(sorted_models, 1):
    print(f"{name:<20} {test_dev:<15.6f} {rank}")

print(f"\nTotal claim predictions:")
print(f"  Observed:      €{df['ClaimAmountCut'].values[test].sum():,.0f}")
print(f"  LGBM (tuned):  €{np.sum(df['Exposure'].values[test] * df_test['pp_t_lgbm_tuned']):,.0f}")

FINAL MODEL COMPARISON

Baseline GLM:
  Train: 74.096244, Test: 72.419647, Ratio: 0.977

GLM + Splines:
  Train: 73.705976, Test: 72.252291, Ratio: 0.980

LGBM (untuned):
  Train: 66.913447, Test: 72.188278, Ratio: 1.079

LGBM (tuned):
  Train: 69.085528, Test: 71.836592, Ratio: 1.040

Model                Test Deviance   Rank
---------------------------------------------
LGBM (tuned)         71.836592       1
LGBM (untuned)       72.188278       2
GLM + Splines        72.252291       3
Baseline GLM         72.419647       4

Total claim predictions:
  Observed:      €9,640,879
  LGBM (tuned):  €9,033,203


## Task 5: Summary

### Results

We successfully trained and tuned a LightGBM model, achieving the best performance:

**Performance Ranking:**
1. LGBM (tuned): 71.84 test deviance
2. LGBM (untuned): 72.19
3. GLM + Splines: 72.25
4. Baseline GLM: 72.42

### Key Insights

1. **Gradient boosting outperforms GLMs** for this dataset
2. **Tuning reduced overfitting**: Overfit ratio improved from 1.079 → 1.040
3. **Best hyperparameters**: learning_rate=0.01, n_estimators=500
   - Slower learning with more trees = better generalization
4. **Feature engineering matters**: GLM improved with splines (72.42 → 72.25)

### Overfitting Check

- Tuned LGBM has ratio of 1.040 (slight overfitting, but acceptable)
- Much better than untuned (1.079)
- Could potentially reduce further with:
  - More regularization (higher `reg_alpha`, `reg_lambda`)
  - Lower `max_depth`
  - More data